In [4]:
!huggingface-cli login

/bin/bash: huggingface-cli: command not found


## Loading Llama2 7b tokenizer and model

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from accelerate import init_empty_weights
from transformers import BitsAndBytesConfig
import torch

# Define the model name and cache directory
model_name = "meta-llama/Llama-2-7b-hf"  # LLaMA 2-7B model
cache_dir = "/scratch/gilbreth/anand173/model_cache"

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Use 4-bit quantization
    bnb_4bit_use_double_quant=True,  # Enable double quantization for memory savings
    bnb_4bit_compute_dtype=torch.bfloat16,  # Use bfloat16 for computation
)

# Load the tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)

# Load the model with 4-bit quantization and device map
print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",  # Automatically allocate model layers across GPU/CPU
    cache_dir=cache_dir,
)

# Ensure the pad token is set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

print("Model and tokenizer loaded successfully!")

Loading tokenizer...
Loading model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model and tokenizer loaded successfully!


## Zero shot example

In [6]:
# Example review to classify
review = "Item was delivered on time and was a direct replcement"

# Format prompt for classification
prompt = f"""### Instruction:
Classify the following review into "Correct Size/Just Right", "Wrong Size", "No Comment". Please respond only with the category:

### Input:
{review}

### Response:"""

# Tokenize the input
inputs = tokenizer(
    prompt,
    return_tensors="pt",
    truncation=True,
    padding=True,
).to("cuda")  # Send input tensors to GPU

# Generate the output
print("Generating classification response...")
outputs = model.generate(
    **inputs,
    max_new_tokens=10,           # Limit the response length
    eos_token_id=tokenizer.eos_token_id
)

# Decode and display the response
response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
print(f"Predicted Label: {response}")

Generating classification response...
Predicted Label: ### Instruction:
Classify the following review into "Correct Size/Just Right", "Wrong Size", "No Comment". Please respond only with the category:

### Input:
Item was delivered on time and was a direct replcement

### Response:
Correct Size/Just Right

##


## Few Shot example

In [11]:
# Few-shot examples for the classification task
few_shot_examples = """
### Examples:
1. Review: "order came quickly and is working fine and is much better price than going to Lowe's or Home Depot to purchase."
   Classification -> No Comment
2. Review: "Perfect Fit - Ideal for when you don't need to replace an otherwise good OEM axle. Fits all FWD/AWD Volvo 850   S/V70 '93-'00"
   Classification -> Correct Size/Just Right
3. Review: "two different ends on cables. doesn't make sense. had to change the end on one side to fit it to the battery."
   Classification -> Wrong Size
"""

# Example review to classify
review = "Have not had to use it yet - but I know how handy it is to have it available. Thanks"

# Format prompt with few-shot examples
prompt = f"""### Instruction:
Classify the following autoparts review into "Correct Size/Just Right", "Wrong Size", "No Comment". Please respond only with the category:

{few_shot_examples}

### Input:
Review: "{review}"
### Response:
"""

# Tokenize the input
inputs = tokenizer(
    prompt,
    return_tensors="pt",
    truncation=True,
    padding=True,
).to("cuda")  # Send input tensors to GPU

# Generate the output
print("Generating classification response...")
outputs = model.generate(
    **inputs,
    max_new_tokens=10,           # Limit the response length
    eos_token_id=tokenizer.eos_token_id
)

# Decode and display the response
response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
print(f"Predicted Label: {response}")

Generating classification response...
Predicted Label: ### Instruction:
Classify the following autoparts review into "Correct Size/Just Right", "Wrong Size", "No Comment". Please respond only with the category:


### Examples:
1. Review: "order came quickly and is working fine and is much better price than going to Lowe's or Home Depot to purchase."
   Classification -> No Comment
2. Review: "Perfect Fit - Ideal for when you don't need to replace an otherwise good OEM axle. Fits all FWD/AWD Volvo 850   S/V70 '93-'00"
   Classification -> Correct Size/Just Right
3. Review: "two different ends on cables. doesn't make sense. had to change the end on one side to fit it to the battery."
   Classification -> Wrong Size


### Input:
Review: "Have not had to use it yet - but I know how handy it is to have it available. Thanks"
### Response:
No Comment

### Input:
Re


## Parsing first 10 reviews from fit.csv

In [22]:
import csv

# Define the file path for the input reviews
input_file = "fit.csv"
output_file = "fit_predictions_first_10.csv"

# Prepare to write results to a new CSV file
with open(output_file, mode="w", newline="") as out_csv:
    writer = csv.writer(out_csv)
    writer.writerow(["ReviewText", "PredictedLabel"])  # Write headers

    # Initialize a counter
    review_count = 0

    # Read and process each review from the input CSV file
    with open(input_file, mode="r") as in_csv:
        reader = csv.DictReader(in_csv)
        for row in reader:
            if review_count >= 10:  # Process only the first 10 reviews
                break

            review = row["ReviewText"]

            # Format the prompt for each review
            prompt = f"""### Instruction:
Classify the following review into one of the categories: "Correct Size/Just Right", "Wrong Size", or "No Comment". Please respond only with the category name.

### Categories:
- "Correct Size/Just Right": The product fits as expected and performs its intended function without issues.
- "Wrong Size": The product does not fit or requires modifications to work correctly.
- "No Comment": The review does not mention size or fitting issues.

### Examples:
1. Review: "order came quickly and is working fine and is much better price than going to Lowe's or Home Depot to purchase."
   Classification -> No Comment
2. Review: "Perfect Fit - Ideal for when you don't need to replace an otherwise good OEM axle. Fits all FWD/AWD Volvo 850   S/V70 '93-'00"
   Classification -> Correct Size/Just Right
3. Review: "two different ends on cables. doesn't make sense. had to change the end on one side to fit it to the battery."
   Classification -> Wrong Size

### Input:
Review: "{review}"
### Response:
"""

            # Tokenize the input
            inputs = tokenizer(
                prompt,
                return_tensors="pt",
                truncation=True,
                padding=True,
            ).to("cuda")  # Send input tensors to GPU

            # Generate the output
            outputs = model.generate(
                **inputs,
                max_new_tokens=10,  # Limit the response length
                eos_token_id=tokenizer.eos_token_id
            )

            # Decode the response and clean it
            response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

            # Extract only the classification label cleanly
            if "Classification ->" in response:
                response = response.split("Classification ->")[-1].split("\n")[0].strip()
            else:
                # If no proper format, default to "No Comment" for robustness
                response = "No Comment"

            # Write the review and predicted label to the output CSV
            writer.writerow([review, response])

            # Increment the counter
            review_count += 1

print(f"Predictions for the first 10 reviews saved to {output_file}.")

Predictions for the first 10 reviews saved to fit_predictions_first_10.csv.


## Accuracy

In [24]:
import csv

# Define file paths
original_file = "fit.csv"
predictions_file = "fit_predictions_first_10.csv"

# Read the original labels for the first 10 reviews
original_labels = {}
with open(original_file, mode="r") as original_csv:
    reader = csv.DictReader(original_csv)
    for idx, row in enumerate(reader):
        if idx >= 10:  # Limit to the first 10 reviews
            break
        review_text = row["ReviewText"].strip()  # Strip any extra whitespace
        original_labels[review_text] = row["FINAL Fit"].strip()  # Map review to its original label

# Read the predicted labels from the predictions file
predicted_labels = {}
with open(predictions_file, mode="r") as predictions_csv:
    reader = csv.DictReader(predictions_csv)
    for row in reader:
        review_text = row["ReviewText"].strip()  # Strip any extra whitespace
        predicted_labels[review_text] = row["PredictedLabel"].strip()  # Map review to its predicted label

# Compare the labels and calculate accuracy
total_reviews = len(original_labels)  # Only consider the first 10 reviews
correct_predictions = sum(
    1 for review, predicted_label in predicted_labels.items()
    if review in original_labels and original_labels[review] == predicted_label
)

accuracy = correct_predictions / total_reviews if total_reviews > 0 else 0
print(f"Total Reviews Processed: {total_reviews}")
print(f"Correct Predictions: {correct_predictions}")
print(f"Accuracy: {accuracy * 100:.2f}%")

Total Reviews Processed: 10
Correct Predictions: 0
Accuracy: 0.00%


In [25]:
import pandas as pd

og = pd.read_csv(original_file)

og

,ReviewID,ReviewText,FINAL Fit
0,1,You will have to remove the window which is ve...,No Comment
1,2,It does what it is supposed to! Sure it does ...,No Comment
2,4,Item was delivered on time and was a direct re...,No Comment
3,5,"This was a really great part, shipped fast, a...",No Comment
4,6,This puller worked getting off a stubborn wipe...,No Comment
...,...,...,...
2250,2496,Best wipers you will ever buy. Mine have laste...,No Comment
2251,2497,A perfect fit in the oem frames on my 08 Fron...,Correct Size/Just Right
2252,2498,Bought a pair of these for one of my cars.I wa...,No Comment
2253,2499,these are heavy duty extra wide wiper refills....,Wrong Size


In [23]:
pred = pd.read_csv(predictions_file)

pred

,ReviewText,PredictedLabel
0,You will have to remove the window which is ve...,Wrong Size
1,It does what it is supposed to! Sure it does ...,Wrong Size
2,Item was delivered on time and was a direct re...,Wrong Size
3,"This was a really great part, shipped fast, a...",Wrong Size
4,This puller worked getting off a stubborn wipe...,Wrong Size
5,"Works ok, really just a quick adapter as the ...",Wrong Size
6,Shipped really fast. I've had it about a mont...,Wrong Size
7,"I don't love this, it's just a battery. But ...",Wrong Size
8,It only last 1 year and i couldn't find where...,Wrong Size
9,"The vendor immediately phoned me, listened ca...",Wrong Size
